### IMPORT DE BIBLIOTECAS

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
from implicit.evaluation import (train_test_split, 
                                 ndcg_at_k, 
                                 AUC_at_k,
                                 mean_average_precision_at_k, 
                                 precision_at_k)

from implicit.nearest_neighbours import bm25_weight

from scipy.sparse import (csr_matrix, 
                          save_npz, 
                          load_npz)
import scipy.stats
import scipy

from matplotlib import pyplot as plt
import seaborn as sns

# from surprise import Dataset, Reader, SVD, SVDpp, KNNWithMeans
# from surprise.model_selection import train_test_split, cross_validate
# from surprise import accuracy
# from surprise.model_selection import cross_validate

# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import pickle
from tqdm import tqdm
import warnings

In [2]:
warnings.filterwarnings('ignore')

### Carregamento de dados de categorias e histórico de clicks

In [3]:
df_de_cat = pd.read_csv(os.getcwd()+'/.csv/new_category_sample.csv') # DataFrame Informativo das categorias

In [4]:
clicks_de = pd.read_csv(os.getcwd()+'\.txt\clicks_de_sample_2.txt', sep = ',', header=0) # df de histórico de clicks

Valores missing na coluna de OfferTitle que serão excluídos.

In [5]:
clicks_de = clicks_de[clicks_de.OfferTitle.isna() == False]

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [6]:
clicks_de.UserId = clicks_de.UserId.astype('category')
clicks_de.OfferId = clicks_de.OfferId.astype('category')

In [7]:
clicks_de['User'] = clicks_de.UserId.cat.codes
clicks_de['Offer'] = clicks_de.OfferId.cat.codes

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [8]:
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

#df_de_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

#df_de_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_de['Cat_clicks'] = clicks_de.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

In [9]:
df_de_cat = df_de_cat[df_de_cat.Country == 'de'].drop('Country',axis =1)

### Merge do Dataframe de categorias com o dataframe de clicks 

In [10]:
clicks_de = clicks_de.merge(df_de_cat, on = 'Category')

### Criação de coluna com o nº total de clicks do usuário e nº total de clicks do produto

In [11]:
clicks_de['UserTotalClicks'] = clicks_de.groupby(by=['UserId'])['OfferId'].transform('count')

In [12]:
clicks_de['ProductClicks'] = clicks_de.groupby(by='OfferId')['UserId'].transform('count')

In [13]:
clicks_de

,UserId,OfferId,OfferViewId,CountryCode,Category,Source,UtcDate,OfferTitle,User,Offer,Cat_clicks,Category_Name,Translate,Ancestor_ID,filha_1,filha_1_name,UserTotalClicks,ProductClicks
0,23d8fe04a6c1ca67e06376bacf2390f8863e86d75717b4...,8b4fbb7e4ef2dc46326861f4f42c7899,8b4fbb7e4ef2dc46326861f4f42c7899-1076982529341...,de,173701,32ff3ac0aba4118d1eb30f5eb6cc58a32c262b5eb0f7be...,2016-06-02 00:26:02.0,Nova Motors Motorroller - Grido 49ccm² 2 Takt,58085,176956,25,Motorräder Roller & Quads,Motorcycle,173701 172601 601,172601.0,Cars and Motorcycles,1,3
1,81f727a6a6e069ac685412c70c0547c14962519d06ec6b...,8b4fbb7e4ef2dc46326861f4f42c7899,8b4fbb7e4ef2dc46326861f4f42c7899-1076982529341...,de,173701,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-01 08:26:13.0,Nova Motors Motorroller - Grido 49ccm² 2 Takt,211356,176956,25,Motorräder Roller & Quads,Motorcycle,173701 172601 601,172601.0,Cars and Motorcycles,1,3
2,fc2ed6a74ab7f9de7b991a860bfdbe84723c258dfb6400...,8b4fbb7e4ef2dc46326861f4f42c7899,8b4fbb7e4ef2dc46326861f4f42c7899-1076981212888...,de,173701,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-02 09:45:31.0,Nova Motors Motorroller - Grido 49ccm² 2 Takt,410290,176956,25,Motorräder Roller & Quads,Motorcycle,173701 172601 601,172601.0,Cars and Motorcycles,1,3
3,89005d39d61f62c6e34cb410e3fb494b16b351be2c1372...,f354f4249a8e2db2b619d4be05e060f0,f354f4249a8e2db2b619d4be05e060f0-107698413815_...,de,173701,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-02 11:04:50.0,Nova Motors Motorroller - GT3,222747,309575,25,Motorräder Roller & Quads,Motorcycle,173701 172601 601,172601.0,Cars and Motorcycles,1,3
4,36e7dcee7ebf6d564c5b0a81785293f36dfcebaf8778c7...,f354f4249a8e2db2b619d4be05e060f0,f354f4249a8e2db2b619d4be05e060f0-1076981391636...,de,173701,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-02 09:44:08.0,Nova Motors Motorroller - GT3,89230,309575,25,Motorräder Roller & Quads,Motorcycle,173701 172601 601,172601.0,Cars and Motorcycles,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692668,698b15c28bb7f3c5250c67a489a63b828effe723046aaf...,19827e02f9a5637923d977bc7abfc6fa,19827e02f9a5637923d977bc7abfc6fa-1076982536503...,de,172301,a4163e1501c4e3fc5fef534c7fd15fc9df5802659acc01...,2016-06-30 12:50:17.0,Kia Rio or similar IN THUNDER BAY,171470,32773,194,Mietwagen,Car Rent,172301 169901 601,169901.0,Travel and Holidays,1,1
692669,0a18d07ba252ea7cb9bd0d7dda61e3745562a083ae224a...,c93411f16814dcbeef9cb09d365080e8,c93411f16814dcbeef9cb09d365080e8-107698411522_...,de,172301,1ca75349ae407fc232ac2c9f34da04f899e78f73a81292...,2016-06-30 14:05:20.0,Ford Fiesta IN MO I RANA,16372,255891,194,Mietwagen,Car Rent,172301 169901 601,169901.0,Travel and Holidays,1,1
692670,39c07b4a3e5d25a5414e33e29dba907314b5122522a6e9...,be6f3224ae550e4c8329c7b24e0f832d,be6f3224ae550e4c8329c7b24e0f832d-107698411522_...,de,172301,1ca75349ae407fc232ac2c9f34da04f899e78f73a81292...,2016-06-30 15:45:01.0,Dacia Logan IN SIBIU,93943,242240,194,Mietwagen,Car Rent,172301 169901 601,169901.0,Travel and Holidays,3,6
692671,bf2bc8d8e2e8496eee59a6379619f2c444b47b6b83c257...,8a98e1e6bbbb6e1054a85e93a4cfe023,8a98e1e6bbbb6e1054a85e93a4cfe023-1076982533033...,de,172301,f6ea7dbba25ddf071177cb05b92679fe7ca1a8b00d770f...,2016-06-30 10:12:58.0,Ford Focus IN HOUSTON,310693,176049,194,Mietwagen,Car Rent,172301 169901 601,169901.0,Travel and Holidays,1,1


In [14]:
#clicks_de_filtered.UserId = clicks_de_filtered.UserId.astype('category')
# clicks_de_filtered.OfferId = clicks_de_filtered.OfferId.astype('category')


# clicks_de_filtered['User'] = clicks_de_filtered.UserId.cat.codes
# clicks_de_filtered['Offer'] = clicks_de_filtered.OfferId.cat.codes#Cap minimo de clicks para integrar o sistema de recomendação
clicks_de_filtered = clicks_de[(clicks_de.ProductClicks > 17) & (clicks_de.ProductClicks < (clicks_de.ProductClicks.mean() + 2*clicks_de.ProductClicks.std()))]

In [15]:
clicks_de_filtered['UserTotalClicks'] = clicks_de_filtered.groupby(by=['UserId'])['OfferId'].transform('count')
clicks_de_filtered = clicks_de_filtered[(clicks_de_filtered.UserTotalClicks > 7)]

In [16]:
clicks_de_filtered.UserTotalClicks

1986       8
2037       8
2038       8
2181       8
3210       8
          ..
690964     8
691311    10
691313    10
691499    12
691568     8
Name: UserTotalClicks, Length: 6193, dtype: int64

In [17]:
clicks_de_filtered.to_csv('de_model_sample.csv')


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [18]:
clicks_per_user_product = clicks_de_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

### Criação de matrizes esparsas Usuário-item e item-usuário

In [19]:
alpha = 40
sparse_item_user = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['Offer'], clicks_per_user_product['User'])))
sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))

sparse_user_item = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
data = (sparse_item_user).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente

In [20]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
item_user_clicks = bm25_weight(data, K1=100, B=0.8)
# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_clicks = item_user_clicks.T.tocsr()

In [21]:
#Esparsidade de matriz
possible_interactions = sparse_item_user.shape[0]*sparse_item_user.shape[1]
interacted = len(sparse_item_user.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999999644388504

 Mais de 99.99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [22]:
save_npz(os.getcwd()+"/.npz/de/sparse_user_item.npz", sparse_user_item)
save_npz(os.getcwd()+"/.npz/de/sparse_item_user.npz", sparse_item_user)

In [23]:
model_path = os.getcwd()+'/.pkl/de/de_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [24]:
offers = pickle.load(open(os.getcwd()+"/.pkl/de/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [25]:
def als_model():
    
    '''computes p@k and map@k evaluation metrics and saves model'''
    
    #sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test = implicit.evaluation.train_test_split(sparse_user_item, train_percentage=0.8)

    model = implicit.als.AlternatingLeastSquares(factors=10, 
                                                 regularization=1, 
                                                 iterations=50,
                                                 calculate_training_loss=False)
    alpha=15
    model.fit(train*alpha)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model

In [26]:
train, test, model = als_model()
implicit.evaluation.precision_at_k(model,train,test)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/408 [00:00<?, ?it/s]

0.3658787255909558

# Avaliação Baseline

In [201]:
def get_k_most_popular(sparse_item_user, k):
    
    """Retorna um array com os códigos dos itens clicados por mais clientes da base de dados"""
    
    new_sparse = sparse_item_user.copy() # Nova esparsa para não modificar a original
    interaction = np.ones_like(new_sparse.data) # Altera os valores da matriz esparsa para '1' se houve interação para que não sejam contadas múltiplas interações do mesmo cliente com o mesmo produto
    new_sparse.data = interaction # Assimila o array de interações de 1's aos dados da matriz esparsa
    pop_items = np.array((new_sparse).sum(axis = 1)).reshape(-1) # Aqui é feita a soma de interações que ocorreram em cada uma das linhas, que correspondem a cada um dos produtos
    most_popular = pd.Series(pop_items).sort_values(ascending = False)[:k] # Os itens são convertidos em série para que possam ser ordenados e então captados os deus com maior incidência para que sejam retornados seus indices(código do produto)
    
    return most_popular.index

 O modelo de baseline considerará como sugestão os produtos mais populares de todo o dataframe como recomendação para todos os clientes

In [202]:
def get_top_k(userid, sparse_user_item, k=10):
    
    '''Retorna os top-K produtos clicados por um usuário de acordo 
    com o nível de confiança obtido através das suas interações'''
    
    top_K =pd.Series(sparse_user_item[userid].data, index = sparse_user_item[userid].indices).sort_values(ascending = False)[:k]
    return top_K.index

In [139]:
def baseline_precision_at_k(sparse_item_user, k=10, test_pct=0.2):

    """Avalia a precisão caso o modelo recomendasse apenas os itens mais populares a todos os clientes.
    A métrica é calculada com base em uma seleção aleatória de 20% dos clientes."""
    
    # Seleção aleatória de 20% dos usuários para testagem por popularidade
    test_sample = np.random.choice(sparse_item_user.indices,
                                   size = int(test_pct*len(sparse_item_user.indices)), 
                                   replace = False) 
    
    most_popular = (get_k_most_popular(sparse_item_user, k)) # lista de 10 itens mais populares
    
    total_precision = 0 
    sparse_user_item = sparse_item_user.T.tocsr()
    
    for user in test_sample:
        relevance = 0
        top_K = get_top_k(user, sparse_user_item, k)
        
        for item in most_popular:
            if item in top_K: # Alteração do pesoa das interações para 0 e 1. Assim, será contabilizado apenas se o usuário interagiu com o item ou não.
                interact = 1
            else:
                interact = 0
            relevance+= interact
            
        relevance/=k
        total_precision+=relevance
        
    total_precision/=len(test_sample)
    
    return total_precision

In [308]:
baseline_precision_at_k(sparse_user_item.T.tocsr())

0.1477638640429337

### Avaliação do modelo

In [31]:
def model_evaluation(train, test, model, k=10): 
    
    '''Avaliação do modelo treinado com as funções da biblioteca Implicit.
    Retorna dicionário com p@k, map@k, ndcg@k e auc@k.'''

    
    p_at_k = implicit.evaluation.precision_at_k(model, train_user_items=train, 
                                                test_user_items=test,
                                                K=k, 
                                                show_progress = False)
    
    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=k, 
                                                             show_progress = False)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=k, 
                                              show_progress = False)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=k, 
                                            show_progress = False)
    metrics = {'p@K':p_at_k, 
               'map@K': m_at_k, 
               'ndcg@K':ndcg_at_k, 
               'auc@K':auc_at_k}
    
    return metrics

In [32]:
metrics = model_evaluation(train, test, model, k = 10)

In [33]:
metrics

{'p@K': 0.361863488624052,
 'map@K': 0.1743745088794595,
 'ndcg@K': 0.23899339127583177,
 'auc@K': 0.672037884709014}

In [323]:
clicks_de_filtered.UserTotalClicks.shape

(3821,)

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [143]:
def recommend(user, K=10):
    
    ''' Retorna uma lista de itens recomendados para o usuário dado de acordo com a biblioteca Implicit.
        Também é retornado uma lista com os itens já clicados por esse usuário'''
    
    #sparse_user_item = load_npz("/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
    
    if user not in sparse_user_item.T.tocsr().indices:
        return "Invalid User"
        
    recommended, scores = (model.recommend(user, sparse_user_item[user], K))

    original_user_items = list(sparse_user_item[user].indices)

    return recommended, original_user_items

In [144]:
x = recommend(402990)

In [440]:
recs= []
for item in x[0]:
    recs.append([item, offers[item], offer_cat[item], offer_filha1[item]])
pd.DataFrame(recs, columns = ['Offer','OfferTitle','Category','Filha_1'])

,Offer,OfferTitle,Category,Filha_1
0,67419,Apple MacBook Pro 15 Retina 2015 (MJLT2D/A),113501,110001
1,58395,Fellowes PrivaScreen privacy filter,113501,110001
2,53742,Fellowes PrivaScreen privacy filter,113501,110001
3,318377,Vogel´s PTS 1214 TabLock protective case for i...,113501,110001
4,104160,Vogel´s PTS 1216 TabLock protective case for i...,113501,110001
5,74523,4You pencil case XXL Shades Purple pencil case,100360623,117901
6,18291,Longoni Cue Wax - Original LONGONI,100014313,163601
7,19258,BARTS Wildbert Beanie,107401,107301
8,77282,MYDAY fleece jacket LORD,108101,107301
9,200362,HP EliteBook 8460p,113501,110001


In [361]:
recs= []
for item in x[1]:
    recs.append([item, offers[item], offer_cat[item], offer_filha1[item]])
pd.DataFrame(recs, columns = ['Offer','OfferTitle','Category','Filha_1'])

,Offer,OfferTitle,Category,Filha_1
0,68,VOGGENREITER THE BIG BOOK FOR SAXOPHONE,100434023,117901
1,7190,PPV Media Self-promotion for musicians,100434023,117901
2,89779,Tipard YouTube Converter,117001,110001
3,228812,HP &quot;15-ac101ng (N9R55EA) 39,113501,110001


In [127]:
def most_similar_items(item_id, n_similar=10):
    '''computes the most similar items'''
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, score = model.similar_items(item_id, n_similar, filter_items = [item_id])

    return similar

In [128]:
a = most_similar_items(32335)

In [129]:
print(offers[32335])
[offers[x] for x in a]

Samsung Galaxy S5 mini white


['Samsung Galaxy S6 edge Black 64GB',
 'Huawei P8lite black',
 'Apple iPhone 5s 16GB Silver',
 'Samsung Galaxy S5 neo black',
 'Ssangyong REXTON W SAPPHIRE 2.0 D Automat.',
 'Ford Kuga Titanium 4x4 Auto.Leather Navi Panorama',
 'Volkswagen Touareg V10 TDI fully equipped',
 'Skoda Fabia 1.6 TDI DPF Ambition 1.HAND CLIMATE SHZ EU-5',
 'HP EliteBook 8460p',
 "BMW M3 E30 Cabrio collector's item rarity"]

In [84]:
def most_similar_users(user_id, n_similar=10):
    
    '''computes the most similar users and which items they have in common with the user'''
    
    sparse_user_item = load_npz(os.getcwd()+"/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, scores = model.similar_users(user_id, n_similar, filter_users = [user_id])

    # original users items
    original_user_items = list(sparse_user_item[user_id].indices)
    common_items_users = {}

    # now we want to add the items that a similar user has rated
    for user in similar:
        # Verifica em cada usuário considerado similar quais são os itens que estes
        # tem em comum com o usuário selecionado
        common_items_users[user] = set(list(sparse_user_item[user].indices)) & set(original_user_items)
    
    # retorna usuários similares, e quais são os itens comuns correspondentes a cada um desses usuários
    return similar, common_items_users

In [87]:
most_similar_users(185847)

(array([136070, 341180, 210811, 386973, 281488, 315187, 404675,  37545,
        212047, 125358]),
 {136070: {65099, 134925, 276934},
  341180: {87420, 174710},
  210811: {174710, 206133, 323181},
  386973: set(),
  281488: {49585, 87420, 173209, 174710},
  315187: set(),
  404675: set(),
  37545: set(),
  212047: {268885},
  125358: set()})

In [88]:
def recalculate_user(user_ratings):
    
    '''adds new user and its liked items to sparse matrix and returns recalculated recommendations
       Receives the user clicked products vector (user_ratings)''' 

    m = load_npz('sparse_user_item.npz')
    n_users, n_movies = m.shape

    ratings = [alpha for i in range(len(user_ratings))]

    m.data = np.hstack((m.data, ratings))
    m.indices = np.hstack((m.indices, user_ratings))
    m.indptr = np.hstack((m.indptr, len(m.data)))
    m._shape = (n_users+1, n_movies)

    # recommend N items to new user
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ =  zip(*model.recommend(n_users, m, recalculate_user=True))
    
    return recommended

In [345]:
import random

def suggestions(user_id, K=500, n_best_seller=2):
    
    '''
    Retorna recomendações segmentadas de acordo com as categorias clicadas pelo usuário passado. 
    À lista de recomendações são adicionados dois top 10 best-sellers da categoria em questão
      
      Inputs: 
       
       user_id -> ID categórico do usuário
       
       K -> Quantidade de sugestões que serão retornadas somente pelo modelo
       
       n_best_seller -> Quantidade de sugestões que serão retornadas por popularidade em cada categoria considerada
       
       Output:
       
       cat_suggestions -> Dicionário com as sugestões agrupadas por categoria já clicada pelo usuário
       
       original -> Array com os clicks originais do usuário selecionado
    '''      
        
    if len(sparse_user_item[user_id].indices) == False:
        return "Invalid User"
    
    
    cat_suggestions = {} ### Dicionario de sugestões agrupadas por categoria. Retornoda função
    recs, original  = recommend(user_id, K) # Resposta original do modelo de recomendações e clicks originais
    pd_recs = (pd.DataFrame((recs, [offer_cat.get(key) for key in recs])).T).rename({0:'Offer', 1:'Category'}, axis = 1) # Dataframe p rastrear a categoria da oferta
    
    
    best_cat = pd.Series(offer_cat[cat] for cat in original).value_counts().index # Filtra as categorias mais clicadadas pelo usuário
    
    for i in range(len(best_cat)): # percorrer as categorias de acordo com os clicks do usuários
        
        if len(pd_recs[pd_recs.Category == best_cat[i]]) != 0: ## Se não houver nenhuma recomendação da categoria em questão, não adicionar à lista

            pop_recs = products_info[products_info.Category==best_cat[i]][:20] # seleciona os 10 itens mais populares
            pop_recs = pop_recs[pop_recs.Offer.isin(original) == False] # Filtra para manter apenas os que ainda não foram clicados

            pop_recs = random.sample(pop_recs.Offer.values.tolist(), pop_recs.shape[0])[:n_best_seller]  # Seleciona 2 ofertas mais populares da categoria

            pop_recs.extend(pd_recs[pd_recs.Category == best_cat[i]].Offer.values[:3].tolist()) #Soma aos best sellers as sugestões do modelo de acordo com a categoria
            cat_suggestions[best_cat[i]] = pop_recs
    
    others = []                     #Verificar sugestões de categorias que nunca foram clicadas pelo usuário
    for row in pd_recs.index[:15]:
        if pd_recs.Category.values[row] not in best_cat.values:
            #print(cat_name[pd_recs.Category.values[row]]) #Ver quais são as categorias que não se encaixaram
            others.append(pd_recs.Offer.values[row])
    random.shuffle(others)
    others = others[:5]
    
    cat_suggestions['Others'] = others
    
    return cat_suggestions, original

In [349]:
a = suggestions(384302, 300, 2)

In [350]:
# Criação de dataframe com as categorias dos produtos recomendados
pd_originals = (pd.DataFrame((a[1], [offer_cat.get(key) for key in a[1]])).T).rename({0:'Offer',1:'Category'}, axis =1) 
 

for category in a[0].keys():
    
    if category == 'Others':
        print("\n",'Achamos que você talvez também goste:',"\n")
        for item in a[0][category]:
            print(offers[item])
        print('\n')
        continue
        
    print('\nPorque você clicou na seção',cat_name[category],"em: \n")
    
    for item in pd_originals[pd_originals.Category == category][:5].Offer.values:
        print(offers[item])
        
    print('\nSugerimos:\n')        
   #print("\n",'#'*5,'Sugestões:','#'*5,"\n")
    
    for item in a[0][category]:
        print(offers[item])
    print('\n')


Porque você clicou na seção Garden furniture em: 

Grizzly Battery 40V
FLORABEST® braided chaise longue
FLORABEST® aluminum pruning shears

Sugerimos:

4rain underground rainwater tank 6800 L
El Fuego Dutch Oven Set 7 pieces
TIP Pumps indoor fountain pump for indoor use WPZ 450 R
4rain underground rainwater tank 6800 L
FLORABEST® long-handle hedge trimmer / pole pruner FHEHS 900 A1



Porque você clicou na seção Power tools em: 

ULTIMATE SPEED® 4 wheel covers
Burg Wächter laser range finder LASER METER 50 PS 7550

Sugerimos:

BAUFIX facade paint
SILVERCREST® Raclette Grill SRG 1200 B2 / SRGL 1200 A1
BM solid wood workbench
Holzmann Maschinen small parts chute rack KSR 94D
Westfalia grinding wheel for chain sharpeners



Porque você clicou na seção drugstore em: 

Spontex floor wiper Easy Max
BAUFIX teak care set

Sugerimos:

Hydrofugal Classic Roll-On
Sonax XTREME rim cleaner PLUS 400 milliliter spray can
Frosch All-Purpose Cleaner Frosch Soda organic quality



Porque você clicou na

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [92]:
df_temp = clicks_de[['Offer','OfferTitle']]

In [93]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [94]:
teste_dicio = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [95]:
#dic = pd.read_csv('de_traduzido.csv')
#dic['Offer2'] = dic['OfferTitle'].map(dict(zip(clicks_de.OfferTitle, clicks_de.Offer)))

In [96]:
pickle.dump(offers, open('.pkl/offers.pkl', 'wb'))

### Criação e armazenamento do dicionário código_oferta -> Filha_1 e código_oferta -> Categoria (carregado na parte de cima do código).

In [97]:
clicks_de.filha_1 = clicks_de.filha_1.astype('int32')

In [98]:
df_temp = clicks_de[['Offer','filha_1', 'Category']]

In [99]:
df_temp = df_temp.drop_duplicates(['Offer','filha_1', 'Category'])

In [100]:
offer_filha1 = dict(zip(df_temp['Offer'], df_temp['filha_1']))
pickle.dump(offer_filha1, open('.pkl/offer_sub1.pkl', 'wb'))

In [101]:
offer_cat = dict(zip(df_temp['Offer'], df_temp['Category']))
pickle.dump(offer_filha1, open('.pkl/offer_cat.pkl', 'wb'))

### Tabela de info dos produtos

In [102]:
products_info = clicks_de[['Offer','Category','filha_1', 'ProductClicks']]

In [103]:
products_info.filha_1 = products_info.filha_1.astype('int32')

In [104]:
products_info = products_info.drop_duplicates().reset_index().drop('index', axis = 1)

In [105]:
products_info.rename({'filha_1':'SuperCat'}, axis = 1, inplace = True)

In [106]:
products_info.sort_values(by=['SuperCat','ProductClicks'], ascending = False, inplace = True)

In [107]:
products_info.to_csv('products_info.csv')

In [108]:
products_info.head()

,Offer,Category,SuperCat,ProductClicks
273416,244858,6513,100471423,76
128250,27702,5101,100471423,61
128358,238298,5101,100471423,53
128034,321905,5101,100471423,46
128654,48922,5101,100471423,43


### Dicionário Nome Sub1

In [109]:
df = clicks_de[['filha_1','filha_1_name']].drop_duplicates()
sub1_name = dict(zip(df.filha_1, df.filha_1_name))
pickle.dump(sub1_name, open('.pkl/sub1_name.pkl', 'wb'))
sub1_name[100471423]

'Entertainment'

### Dicionário Nome Cat

In [110]:
df = clicks_de[['Category','Translate']].drop_duplicates()
cat_name = dict(zip(df.Category, df.Translate))
pickle.dump(cat_name, open('.pkl/cat_name.pkl', 'wb'))
cat_name[6513]

'Video Games'

## Tunagem de hiperparametros

In [214]:
grid = {'factors': [10,50,100,200],
       'regularization':[1,0.5, 0.1, 0.01],
        'iterations':[10,30,50, 100], 
        'alphas':[1,15,40,60, 100]}

In [215]:
model = implicit.als.AlternatingLeastSquares()

In [246]:
def gridsearch_als(grid):
    results = []
    for factor in grid['factors']:
        for regularization in grid['regularization']:
            for iteration in grid['iterations']:
                for alpha in grid['alphas']:
            
                    model = implicit.als.AlternatingLeastSquares(factors = factor, 
                                                                 regularization = regularization, 
                                                                 iterations = iteration)
                    model.fit(train*alpha)

                    partial = model_evaluation(train, test, model, )
                    results.append([factor, regularization, iteration, alpha, partial['p@K'],partial['map@K'], partial['ndcg@K'], partial['auc@K']])
                
    final = pd.DataFrame(results, columns = ['Factors','Regularization','Iteration','alpha','P@K','MAP@K','NDCG@K','AUC@K'])
    return final
                

In [ ]:
grid_results = gridsearch_als(grid)

In [248]:
grid_results.to_csv('params.csv')

In [249]:
grid_results.sort_values(by=['P@K','NDCG@K'], ascending = False)

,Factors,Regularization,Iteration,alpha,P@K,MAP@K,NDCG@K,AUC@K
10,10,1.00,50,1,0.277019,0.118268,0.161464,0.625906
75,10,0.01,100,1,0.277019,0.112387,0.157666,0.626029
30,10,0.50,50,1,0.276451,0.115258,0.158532,0.624036
50,10,0.10,50,1,0.275882,0.117437,0.161001,0.626247
55,10,0.10,100,1,0.270762,0.113639,0.156124,0.623370
...,...,...,...,...,...,...,...,...
4,10,1.00,10,100,0.088168,0.017550,0.035890,0.538423
304,200,0.01,10,100,0.072241,0.019690,0.034520,0.530543
64,10,0.01,10,100,0.072241,0.010721,0.025446,0.530581
144,50,0.01,10,100,0.070535,0.020357,0.033903,0.528062
